# Introducción al Machine Learning con Python
## Ejercicio: Reuters Dataset

Para este ejemplo vamos a utilizar NumPy, matplotlib y keras. Todos estos ya se encuentran disponibles en Google Colaboratory, asique no es necesario instalar nada.

Si quieren seguir el ejemplo de manera local en sus computadoras, les recomendamos utilizar la distribución [Anaconda](https://www.anaconda.com/) como su entorno de Python, para minimizar las complicaciones con la instalación de los paquetes.

Este ejercicio está basado en notebooks de François Chollet disponible en https://github.com/fchollet/deep-learning-with-python-notebooks.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import reuters
from keras import layers, models

Using TensorFlow backend.


In [2]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [4]:
len(train_data), len(test_data)

(8982, 2246)

El dataset reuters consiste en un poco más de 11,000 cables de noticias de la agencia de noticias Reuters. Está codificado de manera que cada palabra ha sido reemplazada por un número. Por como cargamos los datos, vamos a manejar un vocabulario compuesto por las 10,000 palabras más comunes, y las palabras que no entren en esta categoría van a ser representadas por un número particular que indica "palabra desconocida".

Si queremos leer los cables, es necesario traducirlos de vuelta a palabras. Esto se puede hacer mediante el indice de palabras del dataset, y definiendo una función que lo recorra de manera inversa:

In [9]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for key, value in word_index.items()])

def decode_newswire(nw):
    return " ".join(reverse_word_index.get(i - 3, "?") for i in nw)

In [10]:
decode_newswire(train_data[0])

'? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

Una vez cargados los datos, es necesario codificarlos de manera adecuada para que la red neuronal los pueda trabajar. Una de las formas de hacer esto es definiendo dos funciones, una para vectorizar los datos de entrada, y otra para codificar la categoría de cada cable:

In [13]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

La función `vectorize_sequences` se usa para codificar el texto de los cables mediante una representación conocida como _bag of words_. Cada vector resultante tiene 10,000 elementos, con valor 0 o 1 y de esta manera se indica la presencia de una palabra en el cable.

La función `to_one_hot` se ocupa de codificar las etiquetas, y genera un vector de 46 elementos donde todos tienen valor 0, excepto la categoría a la cual corresponde la noticia del cable. La función to_one_hot es equivalente a `keras.utils.to_categorical`.

Una vez definidas estas funciones, debemos convertir los datos a estas representaciones:

In [14]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = to_one_hot(train_labels)
y_test = to_one_hot(test_labels)